<a href="https://colab.research.google.com/github/RuchitaD1/NLP-assignment-3/blob/master/NLP_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download('brown')
nltk.download('universal_tagset')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


In [0]:
brown_sentences = brown.tagged_sents(tagset='universal')

In [0]:
train_sentences = []
train_tags = []
vocabulary = {}

In [0]:
for sentence in brown_sentences:
  words = []
  tags = []
  for t in sentence:
    if t[0] in vocabulary:
      vocabulary[t[0]] += 1
    else:
      vocabulary[t[0]] = 1
    words.append(t[0])
    tags.append(t[1])
  train_sentences.append(words)
  train_tags.append(tags)

In [0]:
train_tags[0]

['DET',
 'NOUN',
 'NOUN',
 'ADJ',
 'NOUN',
 'VERB',
 'NOUN',
 'DET',
 'NOUN',
 'ADP',
 'NOUN',
 'ADJ',
 'NOUN',
 'NOUN',
 'VERB',
 '.',
 'DET',
 'NOUN',
 '.',
 'ADP',
 'DET',
 'NOUN',
 'VERB',
 'NOUN',
 '.']

In [0]:
for word in vocabulary:
  if vocabulary[word] <5:
    rare_words.append(word)

In [0]:
def word_ngram_features(i, words):
  
  if i-1 >= 0:
    wim1 = words[i-1]
  else:
    wim1 = "<s>"
  if i-2 >= 0:
    wim2 =  words[i-2]
  else:
    wim2 = "<s>"
  if i+1 < len(words):
    wip1 = words[i+1]
  else:
    wip1 = "</s>"
  if i+2 < len(words):
    wip2 = words[i+2]
  else:
    wip2 = "</s>"
  return ["prevbigram-"+wim1, "nextbigram-"+wip1, "prevskip-"+wim2, \
          "nextskip-"+wip2, "prevtrigram-"+wim1+"-"+wim2, \
          "nexttrigram-"+wip1+"-"+wip2, "centertrigram-"+wim1+"-"+wip1]
  

In [0]:
def word_features(word, rare_words):
  word_flag = 0
  capital_flag = 0
  number_flag = 0
  hypen_flag = 0
  feat_list = []
  if word not in rare_words:
    feat_list.append("word-"+word)
  if word[0].isupper():
    feat_list.append("capital")
  if any(ch.isdigit() for ch in word):
    feat_list.append("number")
  if any(ch == "-" for ch in word):
    feat_list.append("hyphen")
    
  for j in range(1,5):
    if len(word)>=j:
      feat_list.append("prefix"+str(j)+"-"+word[:j])
      
  for j in range(1,5):
    if len(word)>=j:
      feat_list.append("suffix"+str(j)+"-"+word[-j:]) 
  return feat_list

In [0]:
def get_features(i, words, prevtag, rare_words):
  feat_list1 = word_ngram_features(i, words)
  feat_list2 = word_features(words[i], rare_words)
  feature_list = []
  feature_list.append("tagbigram-"+prevtag.lower())
  for feat in feat_list1:
    feature_list.append(feat.lower())
  for feat in feat_list2:
    feature_list.append(feat.lower())
  return feature_list

In [0]:
# main

train_features = []
i = 0
j = 0
for sentence in train_sentences:
  temp = []
  for i in range(len(sentence)):
    if i!=0:
      prevtag = train_tags[j][i-1]
    else:
      prevtag = "<S>"
    features = get_features(i, sentence, prevtag, rare_words)
    temp.append(features)
  j+=1
  train_features.append(temp)

In [0]:
def remove_rare_features(features, n):
  feat_dict = {}
  for l_l_features in features:
    for l_features in l_l_features:
      for feat in l_features:
        if feat in feat_dict:
          feat_dict[feat] += 1
        else:
          feat_dict[feat] = 1
          
  rare_feats = []
  non_rare_feats = []
  for feat in feat_dict:
    if feat_dict[feat]>n:
      non_rare_feats.append(feat)
    else:
      rare_feats.append(feat)
      
  rare_feats = set(rare_feats)
  non_rare_feats = set(non_rare_feats)
  
  
        

In [0]:
remove_rare_features(train_features, 5)